In [2]:
# 🧠 ECG Image Classifier using HOG + XGBoost

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from skimage.feature import hog
from xgboost import XGBClassifier

# Setup
DATASET_DIR = "ECG Image Data"
CATEGORIES = ["Normal", "Myocardial infarction", "Abnormal heartbeat"]
data, labels = [], []

# Load, resize, normalize, extract HOG
for category in CATEGORIES:
    path = os.path.join(DATASET_DIR, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))
        img = img / 255.0
        features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
        data.append(features)
        labels.append(category)

# Convert to arrays
X = np.array(data)
le = LabelEncoder()
y = le.fit_transform(labels)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6,
                      use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Metrics
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Save model + encoder
os.makedirs("model", exist_ok=True)
with open("model/attr_cm_model.pkl", "wb") as f:
    pickle.dump((model, le), f)


ModuleNotFoundError: No module named 'cv2'